In [56]:
import os
import shutil

In [57]:
#name of raw text must be "YYMMDD_title.txt"
#when daily,title is title of daily article
#when article review, title is title of article
#when book review, title is book title

In [58]:
def getcurrentlist():
    pypath=os.path.dirname(os.path.abspath(__file__))
    return pypath

In [85]:
def check_newfiles(rawd,listxt):
    newlist=[fina for fina in [filename for filename in os.listdir(rawd) if ".txt" in filename] if not fina.startswith('.')]
    with open(listxt) as f:
        noldlist = f.readlines()
    oldlist=[ffi.replace("\n","") for ffi in noldlist]    
    newones=list(set(newlist)-set(oldlist))
    return newones

In [71]:
def replace_link(strl):
    for i in range(strl.count("](")):
        word=strl.split('](')[0].split('[')[-1]
        link=strl.split('](')[1].split(')')[0]
        newstrl='<a href="%s">[%s]</a>'%(link,word)
        strl=strl.replace("[%s](%s)"%(word,link),newstrl)
    while strl.count("***") != 0:
        strl=strl.replace('***','<em><strong>',1)
        strl=strl.replace('***','</strong></em>',1)
    while strl.count("**") != 0:
        strl=strl.replace('**','<strong>',1)
        strl=strl.replace('**','</strong>',1)
    while strl.count("*") != 0:
        strl=strl.replace('*','<em>',1)
        strl=strl.replace('*','</em>',1)
    return strl

In [72]:
def replace_i_b(strl):
    word=strl.split('***')[0].split('[')[-1]
    link=strl.split('](')[1].split(')')[0]
    newstrl='<a href="%s">[%s]</a>'%(link,word)
    strl=strl.replace("[%s](%s)"%(word,link),newstrl)
    return strl

In [62]:
def make_contents(rawtxtpath):
    txtname = os.path.splitext(rawtxtpath.split("/")[-1])[0]
    daten=txtname.split("_")[0]
    articlen=txtname.split("_")[1] 
    category=rawtxtpath.split("/")[-3]
    with open(rawtxtpath) as rawt:
        rt = rawt.readlines()
    rt_replace=[replace_link(li.replace("\n","<br>")) for li in rt]
    pcon = str(rt_replace[0])
    for lin in range(len(rt_replace)):
        if lin == 0:
            pass
        else:
            pcon+=str(rt_replace[lin])   
    
    return category,daten,articlen,pcon,txtname

In [63]:
def make_newhtml(category,date,title,pcon,pypath,txtn):
    ldate="20"+str(date)[0]+str(date)[1]+"/"+str(date)[2]+str(date)[3]+"/"+str(date)[4]+str(date)[5]
    if category == "article_review":
        basehtml = pypath+"/docs/article_review/basehtml/ar_base.html"
        listhtml = pypath+"/docs/article_review.html"
        listxtpath = pypath+"/docs/article_review/arlist.txt"
        newartipath = pypath+"/docs/article_review/ar_%s_%s.html"%(date,title)
        newarelpath = "article_review/ar_%s_%s.html"%(date,title)
        caten = "論文解説"
    elif category == 'book_review':
        basehtml = pypath+"/docs/book_review/basehtml/br_base.html"
        listhtml = pypath+"/docs/book_review.html"
        listxtpath = pypath+"/docs/book_review/brlist.txt"
        newartipath = pypath+"/docs/book_review/br_%s_%s.html"%(date,title)
        newarelpath = "book_review/br_%s_%s.html"%(date,title)
        caten = "書評"
    elif category == "lsit":
        basehtml = pypath+"/docs/list/basehtml/l_base.html"
        listhtml = pypath+"/docs/list.html"
        listxtpath = pypath+"/docs/list/llist.txt"
        newartipath = pypath+"/docs/list/dl_%s_%s.html"%(date,title)
        newarelpath = "list/dl_%s_%s.html"%(date,title)
        caten = "日報"
    else:
        pass
    ltitle = '[%s][%s]%s'%(date,caten,title)
    ltitle2 = '[%s]%s'%(caten,title)
    with open(basehtml) as f:
        article = f.readlines()
    article_replace = [l.replace('titleinsertion', title) for l in [ll.replace("pcontents",pcon) for ll in article]]
    with open(newartipath, mode = 'w') as ff:
        ff.writelines(article_replace)
    with open(listxtpath, mode = "a") as fff:
        fff.write(txtn+"\n")
    with open(listhtml) as f4:
        listht = f4.readlines()
    listht.insert(25,'            <li><a href="%s">%s</a></li>\n'%(newarelpath,ltitle))
    with open(listhtml, mode = 'w') as f5:
        f5.writelines(listht)
    with open(pypath+"/docs/index.html") as f6:
        indexhtml = f6.readlines()
    if len(indexhtml) >= 42:
        del indexhtml[31:33]
    indexhtml.insert(27,"          <dt>%s</dt>\n"%(ldate))
    indexhtml.insert(28,'          <dd><a href="%s">%s</a></dd>\n'%(newarelpath,ltitle2))
    with open(pypath+"/docs/index.html", mode = 'w') as f7:
        f7.writelines(indexhtml)

In [64]:
pypath="/Users/ahiram/GoogleDrive/GitHub/Diary/womohideDiary"
#pypath=getcurrentlist()

In [65]:
artrawd=pypath+("/docs/article_review/raw")
arlistp=pypath+("/docs/article_review/arlist.txt")
brtrawd=pypath+("/docs/book_review/raw")
brlistp=pypath+("/docs/book_review/brlist.txt")
ltrawd=pypath+("/docs/list/raw")
llistp=pypath+("/docs/list/llist.txt")
rawd=pypath+("/docs/rawtxt")

In [13]:
newone=check_newfiles(artrawd,arlistp)
if len(newone) !=0:
    for ne in newone:
        shutil.copy2(os.path.join(artrawd,ne),os.path.join(rawd,ne))
        category,daten,articlen,pcon,ttn = make_contents(os.path.join(artrawd,ne))
        make_newhtml(category,daten,articlen,pcon,pypath,ttn)
newone=check_newfiles(brtrawd,brlistp)
if len(newone) !=0:
    for ne in newone:
        shutil.copy2(os.path.join(brtrawd,ne),os.path.join(rawd,ne))
        category,daten,articlen,pcon,ttn = make_contents(os.path.join(brtrawd,ne))
        make_newhtml(category,daten,articlen,pcon,pypath,ttn)
newone=check_newfiles(ltrawd,llistp)
if len(newone) !=0:
    for ne in newone:
        shutil.copy2(os.path.join(ltrawd,ne),os.path.join(rawd,ne))
        category,daten,articlen,pcon,ttn = make_contents(os.path.join(ltrawd,ne))
        make_newhtml(category,daten,articlen,pcon,pypath,ttn)

In [50]:
asa = 'This is *italic* and **bold** then *** bolditalic*** ***ohayou*** ,[aaaa]and [XXXXX](YYYY),[zzzzzz](rererer).'

In [33]:
asa.split('](')[0].split("[")[-1]

'XXXXX'

In [35]:
asa.split('](')[1].split(")")[0]

'YYYY'

In [36]:
asa.count("](")

2

In [48]:
asa.split("***")[3]

'ohayou'

In [51]:
sa=replace_link(asa)

In [52]:
sa

'This is <em>italic</em> and <strong>bold</strong> then <em><strong> bolditalic</em></strong> <em><strong>ohayou</em></strong> ,[aaaa]and <a href="YYYY">[XXXXX]</a>,<a href="rererer">[zzzzzz]</a>.'

In [86]:
newone=check_newfiles(artrawd,arlistp)

['190416_ブログの自動生成プログラムの作成test1.txt'] n
['', '190416_ブログの自動生成プログラムの作成test1.txt', '190416_ブログの自動生成プログラムの作成test1.txt'] o


In [87]:
newone

[]

In [80]:
a="190416_ブログの自動生成プログラムの作成test1.txt\n"

In [81]:
a.replace("\n","")

'190416_ブログの自動生成プログラムの作成test1.txt'

In [82]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 